# 6. Aggregate clustering <a class="anchor" id="clust"></a>


### Overview

This section of the report involves combining (aggregating) two or more sample datasets. This allows a direct comparison of these samples, rather than analysing them separately.

**IMPORTANT: To run this section, you must have processed all the samples you wish to combine in the '5. Filtering cells and clustering' section. This means that outliers and non-target cells have already been identified and removed. Section 5 only has to run once for each sample, as it outputs a datafile for each sample that is imported into this section.**

**ALSO IMPORTANT: Aggregate datasets can be very large and utilise a lot of memory. You can see at the bottom of the page the memory usage ('Mem: xxx/xxxMB'). Keep an eye on this. If you use all available memory the server may crash.**

*************************************

## Contents

[6a. Initial setup](#setupag)

[6b. Choose samples to aggregate and import data](#sampleag)

[6c. Processing expression data (dimensionality reduction)](#dimag)

[6d. PCA, UMAP and t-SNE plots (plotting dimenstionality reduction data)](#dimplotag)

[6e. Visualise gene expression by marker](#plotag)

[6f. Differential expression between samples](#deag)

**************************

## <font color="green">6a. Initial setup</font> <a class="anchor" id="setupag"></a>

<font color="green">**Each section is designed to be run independently, therefore there is some repeated setup code that needs to be run first. That code is within this subsection, indicated by green text.**</font>

<font color="green">Choose which dataset you want to work on by clicking on one of the setwd() commands below. This sets the working directory for your dataset of choice.</font>

In [ ]:
setwd("~/Fazeleh/Dataset1/scDATA")

In [ ]:
setwd("~/Fazeleh/Dataset2/scDATA")

<font color="green">Load the R packages required for this section. If packages are already installed they can be used simply by loading them with the `library()` function.</font>

In [ ]:
library(ggplot2)
library(tidyverse)
library(viridis)

<font color="green">Install R packages required for this section. Packages not installed on the server need to be installed first, then loaded with `library()`.</font>

<font color="green">Seurat (https://satijalab.org/seurat/) is the main package we will be using in this analysis workflow. Seurat installs multiple dependencies, so you may need to wait a few minutes for installation to complete.</font>

In [ ]:
install.packages("Seurat")
install.packages("patchwork")
library(Seurat)
library(patchwork)

<font color="green">Define a set of colours for plotting. Some of these plots have multiple clusters and it's difficult to find enough contrasting colours to visually separate the clusters. I've developed a set of 25 colours that I've found contrast well, that we can use in the plots for this (and other) sections.</font>

In [ ]:
c25 <- c(
  "dodgerblue2", "#E31A1C", # red
  "green4",
  "#6A3D9A", # purple
  "#FF7F00", # orange
  "black", "gold1",
  "skyblue2", "#FB9A99", # lt pink
  "palegreen2",
  "#CAB2D6", # lt purple
  "#FDBF6F", # lt orange
  "gray70", "khaki2",
  "maroon", "orchid1", "deeppink1", "blue1", "steelblue4",
  "darkturquoise", "green1", "yellow4", "yellow3",
  "darkorange4", "brown"
)

<font color="green">Set the default width and height for plots output on this Notebook. You can modify this as you prefer. Note that every plot in this Notebook is followed by code to output it as a file and this code defines width/height separately from the options below.</font>

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)

***********************************

## 6b. Choose samples to aggregate and import data <a class="anchor" id="sample2"></a>

**IMPORTANT: As mentioned in the Overview section, you need to have run section 5 at least once for your chosen sample. Section 5 outputs a data file of your clustered, quality filtered cells, that will now be imported below.

Running your sample through section 5 created a datafile called 'sample_name_seurat_filtered.rds', so if your sample was called 'liver', the file would be 'liver_seurat_filtered.rds'.

These datafiles will be in your working directories. You can see which samples you've run through section 5 (and thus have generated the required output files) by running the `dir()` command below:

In [ ]:
dir(pattern = "seurat_filtered.rds")

Choose the sample names you wish to work with from the list above (just the names, without the '_seurat_filtered.rds').

Add or remove samples as needed to the `samples` object below:

In [ ]:
samples <- c("Pia", "Choroid")

Give your aggregate samples a name (for example, "high_low"). This is for naming plots and such.

In [ ]:
sample <- "Pia_Choroid"

Now import the data files for those samples.

In [ ]:
samplist <- list()
for (i in 1:length(samples)){
    assign(samples[i], readRDS(paste0(samples[i], "_seurat_filtered.rds")))
    samplist[[i]] <- get(samples[i])
    }

Then merge these datasets (note that this tags your cells with your sample names):

In [ ]:
data <- merge(samplist[[1]], y = samplist[[2:length(samplist)]], add.cell.ids = samples, project = sample)

See a summary of your data object (note that 'samples' means cells and 'features' means genes):

In [ ]:
data

This should contain the combined number of cells from all your samples. You can see the number of cells in each sample by typing the sample name below.

In [ ]:
Pia

**********************************

## 6c. Processing expression data (dimensionality reduction) <a class="anchor" id="dimag"></a>

This is essentially a repeat of section 5, so see that section for details. The following steps are required to generate dimensionality reduction analysis (PCA, T-SNE and UMAP) for the aggregate data.

**NOTE: this regenerates clustering based on the combined data. Any plots and analysis in this section are therefore based on this, rather than the individual sample clustering. If you wish to compare clustering per sample, you'll need to use the results in section 5, where you analysed each sample separately.**

In [ ]:
# Normalise data
data1 <- NormalizeData(data)
# Identification of variable features
data1 <- FindVariableFeatures(data1, selection.method = "vst", nfeatures = nrow(data1))
# Scaling the data
all.genes <- rownames(data1)
data1 <- ScaleData(data1, features = all.genes)
# Perform linear dimensional reduction (PCA)
data1 <- RunPCA(data1, features = VariableFeatures(object = data1))

The above normalises and scales the data, finds variable features and does the initial dimensionality reduction. Now we can run t-SNE and UMAP reduction.

In [ ]:
data1 <- RunUMAP(data1, dims = 1:3, verbose = F)
data1 <- RunTSNE(data1, dims = 1:3, verbose = F)

As in section 5, we can view the top variable genes, but this time for the combined samples. See section 5 for details on modifying the following plot:

In [ ]:
# Identify the 10 most highly variable genes
top_genes <- head(VariableFeatures(data1), 10)
# plot variable features with labels
p <- VariableFeaturePlot(data1, pt.size = 2, cols = c("black", "firebrick"))
p <- LabelPoints(plot = p, points = top_genes, repel = TRUE) +
theme_bw() +
theme(text = element_text(size = 17))
p

Export as a 300dpi tiff

Note that for every plot you can adjust the width and height as needed (both currently 20cm by default). Generate the plot, view it, then you can re-adjust the width/height as needed.

In [ ]:
tiff_exp <- paste0(sample, "_top_genes.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(sample, "_top_genes.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

Now we can visualise your expression data using all 3 dimensionality reduction methods

******************************************

## 6d. PCA, UMAP and t-SNE plots (plotting dimenstionality reduction data) <a class="anchor" id="dimplotag"></a>

### Generate the PCA plot

In [ ]:
p <- DimPlot(data1, reduction = "pca", group.by = 'orig.ident', pt.size = 2, cols = alpha(c25[1:length(samples)],0.5)) + 
theme_bw() +
theme(axis.title=element_text(size=16), axis.text=element_text(size=14))
p

Or split instead?

In [ ]:
p <- DimPlot(data1, reduction = "pca", split.by = 'orig.ident', group.by = 'orig.ident', pt.size = 2, cols = alpha(c25[1:length(samples)],0.5)) + 
theme_bw() +
theme(axis.title=element_text(size=16), axis.text=element_text(size=14))
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(sample, "_PCA_pre_filtration.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(sample, "_PCA_pre_filtration.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

### Generate the UMAP pplot

In [ ]:
p <- DimPlot(data1, reduction = "umap", group.by = 'orig.ident', pt.size = 2, cols = alpha(c25[1:length(samples)], 0.5)) + 
theme_bw() +
theme(legend.position="none", axis.title=element_text(size=16), axis.text=element_text(size=14))
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(sample, "_umap_pre_filtration.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(sample, "_umap_pre_filtration.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

### Generate the tSNE plot

In [ ]:
p <- DimPlot(data1, reduction = "tsne", group.by = 'orig.ident', pt.size = 2, cols = alpha(c25[1:length(samples)], 0.5)) + 
theme_bw() +
theme(legend.position="none", axis.title=element_text(size=16), axis.text=element_text(size=14))
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(sample, "_tsne_pre_filtration.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(sample, "_tsne_pre_filtration.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

*********************************************

## 6e. Visualise gene expression by marker <a class="anchor" id="plotag"></a>

This section allows you to produce a side-by-side visualisation of gene expression for your aggregated samples, for specific genes.

### Dimensionality plots

Here you can generate PCA, UMAP or t-SNE plots for selected markers, with side-by-side plots for each sample.

Enter which markers you want to examine (can be one or many markers): 

In [ ]:
markers <- c("P2ry12", "Tmem119", "Itgam")

Select which type of plot you want to generate ("pca", "umap" or "tsne"):

In [ ]:
redplot <- "pca"

Then generate the plot.

In [ ]:
p <- FeaturePlot(data1, features = markers, reduction = redplot, cols = c("lightgrey", "red"), split.by = 'orig.ident', pt.size = 1)
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(sample, "_", redplot, "_markers_filtered.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(sample, "_", redplot, "_markers_filtered.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

### Violin plot

Here you can visualise the relative expression of markers on each cluster. 

**NOTE: This plot is specifically for comparing 2 samples. This plot may not be informative if you have more than 2 samples in your aggregte dataset**

Generate the plot:

In [ ]:
VlnPlot(data1, features = markers, split.plot = TRUE, split.by = 'orig.ident', cols = c25[1:length(samples)])

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(sample, "_markers_violin.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(sample, "_markers_violin.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

### Heatmap

You can also visualise differences in marker expression between samples with a heatmap:

In [ ]:
p <- DoHeatmap(data1, features = markers, raster = T, group.by = 'orig.ident') + 
scale_fill_gradientn(colors = c("darkorange", "floralwhite", "dodgerblue4")) + 
theme(text = element_text(size = 16)) + labs(color = "Dose (mg)")
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(sample, "_hmap_markers.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(sample, "_hmap_markers.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

**********************************

## 6f. Differential expression between samples <a class="anchor" id="deag"></a>

In the next section (section 7) we examine differential expression between clusters in individual samples. In this section we examine differential expression between samples.

You will need to choose two samples to compare, based on your aggregated dataset. To see which samples you've aggregated:

In [ ]:
samples

Now select the first sample (which will be the baseline sample):

In [ ]:
desamp1 <- "Pia"

Then select the second (comparison) sample:

In [ ]:
desamp2 <- "Choroid"

Then run the Seurat differential expression function:

In [ ]:
DE_genes <- FindMarkers(data1, group.by = 'orig.ident', ident.1 = desamp1, ident.2 = desamp2, logfc.threshold = 0.2)

We can see how many 'differentially expressed' genes this produced by:

In [ ]:
nrow(DE_genes)

Note the `logfc.threshold = 0.2` parameter above. This only tests genes with at least 0.2 log fold difference in expression and speeds up the analysis considerably. But it could also remove some significant DE genes.

To test for this, look at the bottom 6 genes (ordered by p value) by running the `tail(DE_genes)` command below. If these genes are all non-significant (i.e. p_val_adj > 0.05) then you have captured all significant genes. If all these genes are significant (p_val_adj < 0.05) then re-run `FindMarkers()` with `logfc.threshold = 0.1`. This will take much longer to run, but should then capture all DE genes (if not, reduce `logfc.threshold = 0`, but will take a very long time to run).

In [ ]:
tail(DE_genes)

You can view the top 20 DE genes by:

In [ ]:
head(DE_genes, 20)

You can extract just the **significantly** differentially expressed genes (adjusted p < 0.05) like so:

In [ ]:
DE_genes_sig <- DE_genes[DE_genes$p_val_adj < 0.05, ]

See how many significantly DE genes there are

In [ ]:
nrow(DE_genes_sig)

You can also extract genes based on log fold change as well. Enter a log fold change threshold here:

In [ ]:
lfc_threshold <- 0.3

Then filter your data by this log fold change threshold.

In [ ]:
DE_genes_sig_lfc <- DE_genes_sig[DE_genes_sig$avg_log2FC < -lfc_threshold | DE_genes_sig$avg_log2FC > lfc_threshold, ]

Then see how many sig DE genes remain after lfc filtration:

In [ ]:
nrow(DE_genes_sig_lfc)

**NOTE** You can easily filter out all your results here by using too high a lfc threshold. You can see the maximum log fold change for all DE genes like so:

In [ ]:
max(DE_genes$avg_log2FC)

And you can view the minimum as well. Use this min/max log fold change information to decide on a suitable log fold change threshold. This can vary greatly, depending on your data.

In [ ]:
min(DE_genes$avg_log2FC)

You can export the DE genes table to your working directory as a csv file (and then view the entire table in Jupyter by double clicking on the csv file):

In [ ]:
write.csv(DE_genes_sig_lfc, paste0("DE_", desamp1, "_vs_", desamp2, ".csv"))

### Visualising DE results

There are a variety of ways to visualise your DE results. Below are a few examples and more can be added to this workflow as needed.

Not all of these methods have the space to plot all DE genes, so we can provide them with a list of DE genes to plot:

In [ ]:
plotgenes <- rownames(DE_genes_sig)[1:20]

The above pulls out the top 10 most significantly DE genes, by adjusted p value.

You can enter selected genes to plot (e.g., choose genes of interest from the table of DE genes), in which case you'd change the above code to a vector of gene IDs, e.g. `plotgenes <- c("pDC", "Eryth", "Mk", "DC")`. You can include as many genes as you like.

**Scatter plot of individual genes**

You can select `reduction =` to be either `"pca"` for PCA plot, or `"umap"` or `"tsne"`. You can also change the colours or point sizes.

In [ ]:
p <- FeaturePlot(data1, features = plotgenes[1:10], cols = c("lightgrey", "red"), reduction = "pca", pt.size = 1)
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(desamp1, "_vs_", desamp2, "_DE_genes.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(desamp1, "_vs_", desamp2, "_DE_genes.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

**Dot plot**

In [ ]:
p <- DotPlot(data1, features = plotgenes, group.by = 'orig.ident', dot.scale = 12, cols = c("lightgrey", "red")) + RotatedAxis() +
ylab("Sample") + xlab("Genes") +
theme(text = element_text(size = 18))
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(desamp1, "_vs_", desamp2, "_DE_genes_dotplot.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(desamp1, "_vs_", desamp2, "_DE_genes_dotplot.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

**Violin plots of individual genes**

In [ ]:
p <- VlnPlot(data1, features = plotgenes[1:10], split.plot = TRUE, split.by = 'orig.ident', cols = c25, pt.size = 0)
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(desamp1, "_vs_", desamp2, "_DE_genes_violin.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(desamp1, "_vs_", desamp2, "_DE_genes_violin.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

**Heat map**

In [ ]:
p <- DoHeatmap(data1, features = plotgenes, raster = T, group.by = 'orig.ident') + 
scale_fill_gradientn(colors = c("darkorange", "floralwhite", "dodgerblue4")) + 
theme(text = element_text(size = 16))
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(desamp1, "_vs_", desamp2, "_DE_genes_heatmap.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(desamp1, "_vs_", desamp2, "_DE_genes_heatmap.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

[Click here to go to the next section: Differential expression](./scRNASeq_7_DE.ipynb)